<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Germán Urrea
- Nombre de alumno 2: - 


### **Link de repositorio de GitHub:** `https://github.com/G-Urrea/labs_mds_german_urrea`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=1bfab3f3dcb148d23ac6e8daeb6d080eeb518412a7d807f37588e2424a99d8e3
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gdown
url_comics = "https://drive.google.com/uc?id=1zW_o-s3679tM2-2ZDSyHdVy0mT6Etxta"
url_no_label = "https://drive.google.com/uc?id=1zPZWkN6kFQTGWcDzdvo2N-sixK9v-G9T"
gdown.download(url_comics, 'df_comics.csv')
gdown.download(url_no_label, 'comics_no_label.csv')

Downloading...
From: https://drive.google.com/uc?id=1zW_o-s3679tM2-2ZDSyHdVy0mT6Etxta
To: /content/df_comics.csv
100%|██████████| 8.15M/8.15M [00:00<00:00, 138MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zPZWkN6kFQTGWcDzdvo2N-sixK9v-G9T
To: /content/comics_no_label.csv
100%|██████████| 215k/215k [00:00<00:00, 29.1MB/s]


'comics_no_label.csv'

In [4]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [6]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [7]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [8]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [11]:
# df_comics, df_comics_no_label
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

In [12]:
preprocess = ColumnTransformer(
    [('minmax', MinMaxScaler(), atributos_de_interes),
     ('bog', CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2)), "history_text" )
     ],
    # No se utilizarán otras caracteristicas
    remainder = 'drop'
)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [13]:
y = df_comics['alignment']
x = df_comics.drop(columns = ['alignment'])

In [14]:
# Revisar balance de datos
y.value_counts()

Good       743
Bad        429
Neutral    113
Name: alignment, dtype: int64

In [106]:
baseline = Pipeline(
    [('preprocess', preprocess),
     ('Feature Selection', SelectPercentile(score_func = f_classif, percentile=90)),
     ('clf', MultinomialNB())
    ]
)

In [107]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, stratify=y)

In [108]:
baseline.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('minmax', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('bog',
                                                  CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x7f9cb6a7c520>),
                                                  'history_text')])),
                ('Feature Selection', SelectPercentile(percentile=90)),
                ('clf', MultinomialNB())])

In [109]:
base_predict = baseline.predict(x_test)

In [45]:
from sklearn.dummy import DummyClassifier

In [111]:
dummy  = DummyClassifier(strategy = 'stratified', random_state=42).fit(x_train, y_train)

In [110]:
print("Resultados Baseline:")
print(classification_report(y_test, base_predict))

Resultados Baseline:
              precision    recall  f1-score   support

         Bad       0.55      0.58      0.57       129
        Good       0.71      0.78      0.75       223
     Neutral       0.33      0.06      0.10        34

    accuracy                           0.65       386
   macro avg       0.53      0.47      0.47       386
weighted avg       0.63      0.65      0.63       386



In [112]:
print("Resultados Dummy:")
print(classification_report(y_test, dummy.predict(x_test)))

Resultados Dummy:
              precision    recall  f1-score   support

         Bad       0.32      0.34      0.33       129
        Good       0.58      0.55      0.56       223
     Neutral       0.05      0.06      0.06        34

    accuracy                           0.44       386
   macro avg       0.32      0.32      0.32       386
weighted avg       0.45      0.44      0.44       386



### Análisis

Como mínimo estandar de calidad, el clasificador Naive Bayes obtiene métricas significativamente mejores para las clases `Good` y `Bad`, quedandose corto en la clase `Neutral`. Este desbalance en las métricas se puede explicar por el desbalance de clases existente, en donde, entre más muestras tiene la clase, mejores resultados tiene. Este comportamiento también lo presenta el clasificador Dummy, debido a la estrategia utilizada.

De esta primera aproximación se puede evidenciar que el baseline tiene un rendimiento deficiente en las clases minoritarias, especialmente en la predicción de entes neutrales. Aún así, es rendimiento es mejor que un clasificador Dummy, por lo que se puede tomar como una base para mejorar.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [23]:
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV

Se escogen 3 clasificadores, utilizando como guía las recomendaciones de [sklearn](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html):

- **Linear SVC**: Kernel lineal de SVM, recomendado como primera aproximación a la clasificación. Se variará el parámetro de regularización $C$.
- **Naive Bayes**: Recomendado en los casos que se tiene texto como data. Si bien ya se probó en el baseline, podría ser útil probar otros hiperparámetros. Se variará el parámetro de regularización $\alpha$.
- **Random Forest**: Clasificador de ensamblaje. En este caso se escoge por ser uno de los clasificadores de ensamblaje más populares y eficientes con datos de alta dimensionalidad. Se variará el número de estimadores.

Por otro lado, para CountVectorizer no se probarán más parámetros además de los sugeridos, principalmente porque este de por sí toma bastante tiempo en ejecutarse.

In [66]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
preprocess = ColumnTransformer(
    [('minmax', MinMaxScaler(), atributos_de_interes),
     ('bog', CountVectorizer(tokenizer= StemmerTokenizer()), "history_text" )
     ],
    # No se utilizarán otras caracteristicas
    remainder = 'drop'
)

params = [
    {
        'clf': LinearSVC(random_state=42),
        'clf__C': [0.1, 1, 10],
        'selection__percentile' : [20, 40, 60, 80],
        'preprocess__bog__ngram_range': [(1,1), (1,2), (1,3)] 
    },
    {
        'clf' : RandomForestClassifier(random_state = 42),
        'clf__n_estimators': [10, 50, 100],
        'selection__percentile' : [20, 40, 60, 80],
        'preprocess__bog__ngram_range': [(1,1), (1,2), (1,3)] 
    },

    {
        'clf' : MultinomialNB(),
        'clf__alpha': [0.1, 1, 10],
        'selection__percentile' : [20, 40, 60, 80],
        'preprocess__bog__ngram_range': [(1,1), (1,2), (1,3)]
      
    }
]

result=[]

for param in params:

        # Obtener modelo inicializado
        clf = param['clf']
        # Quitar del diccionario
        param.pop('clf')

        #pipeline
        steps = [
            ('preprocess', preprocess), 
             ('selection', SelectPercentile(score_func = f_classif)),
              ('clf', clf)
          ]
        # GridSearch
        grid = HalvingGridSearchCV(Pipeline(steps), param_grid=param, cv=2, verbose = 2, n_jobs = 4,
                                   error_score=0, scoring = 'accuracy')
        grid.fit(x_train, y_train)

        result.append\
        (
            {
                'grid': grid,
                'classifier': grid.best_estimator_,
                'best score': grid.best_score_,
                'best params': grid.best_params_,
                'cv': grid.cv
            }
        )

In [72]:
from operator import itemgetter
# Obtener mejor estimador en base a score
result = sorted(result, key=itemgetter('best score'), reverse=True)

In [136]:
print('Mejor accuracy (CV):')
print(result[0]['best score'])
print('Mejores hiperparámetros')
print(result[0]['best params'])
print(f"Número de features (input SVC): {result[0]['classifier'].named_steps['clf'].n_features_in_}")

print(f'Features de entrada (baseline NB): {baseline.named_steps["clf"].n_features_in_}')

Mejor accuracy (CV):
0.6067415730337078
Mejores hiperparámetros
{'clf__C': 1, 'preprocess__bog__ngram_range': (1, 1), 'selection__percentile': 40}
Número de features (input SVC): 7788
Features de entrada (baseline NB): 17406


El mejor accuracy en el conjunto de entrenamiento lo obtiene el clasificador `Linear SVC` con $C=1$, con un accuracy de 0.6 en el conjunto de validación . Por otro lado el preprocesamiento que mejor se adapta a este es un BoW con unigramas, y una selección del 40% de caracteristicas.

Una posible explicación a que este sea el mejor modelo, es que el parámetro de regularizacion $C$ escogido es el intermedio, por lo que las otras opciones exploradas, o daban mucha holgura, o eran muy estrictas. También resalta que el mejor rendimiento se diera con unigramas, lo más probable es que baste con ciertas palabras clave para diferenciar mejor a heroes de villanos.



In [119]:
grid_clf = result[0]['grid']
print('Linear SVC (Test):')
print(classification_report(y_test, grid_clf.predict(x_test))) 

Linear SVC (Test):
              precision    recall  f1-score   support

         Bad       0.90      0.81      0.85       129
        Good       0.91      0.95      0.93       223
     Neutral       0.78      0.82      0.80        34

    accuracy                           0.89       386
   macro avg       0.86      0.86      0.86       386
weighted avg       0.89      0.89      0.89       386



Probando el mejor clasificador obtenido con el conjunto de prueba, es notable que el accuracy obtenido es de 0.89, por lo que en resultados generales supera al baseline. En particular, es de notar que en general los resultados mucho mejores para las clases minoritarias, por lo que ha mejorado significativamente la capacidad de generalizacion, probablemte gracias al uso de un clasificador distinto y a la reducción del número de caracteristicas de entrada. 


## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

Se predecirán solamente las etiquetas para las muestras de interés, se comparará el clasificador obtenido con Grid Search con el baseline.

In [98]:
# Se obtienen las caracteristicas para Vergil, Gorilla Girl y Batcow
df_prediction = df_comics_no_label[df_comics_no_label.name.isin(['Vergil', 'Gorilla Girl', 'Batcow'])]
df_prediction = df_prediction.reset_index(drop=True).loc[:2, ['name', 'history_text'] + atributos_de_interes]
df_prediction

,name,history_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score
0,Batcow,Bat-Cow was originally a cow that was found by...,70,10,25,20,10,20
1,Gorilla Girl,A carnival performer with the ability to turn ...,90,35,60,60,45,100
2,Vergil,"Vergil, later also known as Nelo Angelo, is on...",90,75,95,90,100,100


In [121]:
grid_clf.predict(df_prediction)

array(['Good', 'Good', 'Good'], dtype=object)

In [115]:
baseline.predict(df_prediction)

array(['Good', 'Bad', 'Neutral'], dtype='<U7')

Ambos clasificadores difieren respecto a `Gorilla Girl` y `Vergil`, es interesante notar que el clasificador obtenido por GridSearch sólo predice la etiqueta `Good`, lo que no es raro tomando en cuenta el desbalance de clases existente en el entrenamiento.

Por último, ambos clasificadores coinciden en que `Batcow` es un ser benevolo, por lo que no sería de extrañar que este efectivamente lo sea.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>